In [816]:
%reload_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell

from data_loader import get_pdf_filepaths, load_page_and_line_indexes, load_split_data, load_raw_indexes_list
from pdf_reader import parse_pdf
from data_cleaner import clean_initial_indexes, add_split_data
from data_saver import save_page_and_line_indexes, save_split_data, save_raw_indexes_list, save_aggregated_data
from data_transformer import (
  get_candidates_and_frequencies, 
  add_frequencies_column, 
  add_is_in_toc, 
  add_importance,
  add_position_in_context,
  add_is_named_entity,
  add_length_of_word,
  add_is_named_author,
  add_tfidf,
  get_raw_indexes_list,
  add_is_in_index,
  aggregate_by_candidate
)


#InteractiveShell.ast_node_interactivity = "all"

# Configuration variables

In [33]:
DATA_DIR_PATH = "../data/"
PDF_SOURCE_DIR_PATH = DATA_DIR_PATH + "pdf/"
PROCESSED_DATA_DIR_PATH = DATA_DIR_PATH + "processed/"

# Load data

In [34]:
file_paths = get_pdf_filepaths(PDF_SOURCE_DIR_PATH)
file_paths

['/Users/bertagrim/dev_projects/final_project/index_generator/data/pdf/2017_Book_TheDataScienceDesignManual.pdf',
 '/Users/bertagrim/dev_projects/final_project/index_generator/data/pdf/morris_07.pdf',
 '/Users/bertagrim/dev_projects/final_project/index_generator/data/pdf/lyons_96.pdf',
 '/Users/bertagrim/dev_projects/final_project/index_generator/data/pdf/2015_Book_LinearAlgebra.pdf',
 '/Users/bertagrim/dev_projects/final_project/index_generator/data/pdf/christensen_04.pdf',
 '/Users/bertagrim/dev_projects/final_project/index_generator/data/pdf/keefe_00.pdf',
 '/Users/bertagrim/dev_projects/final_project/index_generator/data/pdf/linnebo_18.pdf',
 '/Users/bertagrim/dev_projects/final_project/index_generator/data/pdf/oliver_13.pdf',
 '/Users/bertagrim/dev_projects/final_project/index_generator/data/pdf/jenkins_08.pdf',
 '/Users/bertagrim/dev_projects/final_project/index_generator/data/pdf/zimmermann_13.pdf',
 '/Users/bertagrim/dev_projects/final_project/index_generator/data/pdf/shapiro_9

# Raw line and page data


In [837]:

for file_path in file_paths[16:17]:
    raw_line_and_page_indexes = parse_pdf(file_path)

    line_and_page_indexes = clean_initial_indexes(raw_line_and_page_indexes)

  # line_and_page_indexes['file_name']
  # line_and_page_indexes['by_line'].head()
  # line_and_page_indexes['by_page'].head()

    save_page_and_line_indexes(
     processed_data_dir_path=PROCESSED_DATA_DIR_PATH, 
     line_and_page_indexes=line_and_page_indexes
    )


# Data set split


In [838]:
file_path = file_paths[16]

line_and_page_indexes = load_page_and_line_indexes(
  processed_data_dir_path=PROCESSED_DATA_DIR_PATH,
  pdf_filepath=file_path
)
with_split_data = add_split_data(
  file_path=file_path,
  line_and_page_indexes=line_and_page_indexes
)

# with_split_data['by_page_body'].head()

save_split_data(
  processed_data_dir_path=PROCESSED_DATA_DIR_PATH,
  split_data=with_split_data  
)
 


# Prepare input data frames

In [839]:
with_split_data['by_page_toc']

,content,page_number,real_page_num,section_level_1,section_level_2,section_level_3,clean_content
0,Contents\n\nPreface\nIntroduction xi\n1\n\n1 The early history of sorites paradoxes\n1.1 The first sorites\n1.2 Chrysippan silence\n1.3 Sorites arguments and Stoic logic\n1.4 The sorites in later antiquity\n1.5 The sorites after antiquity\n\n8\n8\n12\n22\n27\n31\n\n2 The ideal of precision\n2.1 The emergence of vagueness\n2.2 Frege\n2.3 Peirce\n2.4 Russell\n\n36\n36\n37\n46\n52\n\n3 The rehabilitation of vagueness\n3.1 Vagueness and ordinary language\n3.2 The Black–Hempel debate\n3.3 Family resemblances\n3.4 Open texture\n\n70\n70\n73\n84\n89\n\n4 Many-valued logic and degrees of truth\n4.1 Overview\n4.2 Truth-functionality\n4.3 Three-valued logic: beginnings\n4.4 Three-valued logic: Halldén\n4.5 Three-valued logic: Körner\n4.6 Three-valued logic: second-order vagueness\n4.7 Continuum-valued logic: a rationale\n4.8 Continuum-valued logic: truth-tables\n\n96\n96\n97\n102\n103\n108\n111\n113\n114,7,7,Contents,,,content preface introduction xi early history sorites paradox first sorites chrysippan silence sorites argument stoic logic sorites later antiquity sorites antiquity ideal precision emergence vagueness frege peirce russell rehabilitation vagueness vagueness ordinary language debate family resemblance open texture many-valued logic degree truth overview truth-functionality three-valued logic beginning three-valued logic halldén three-valued logic körner three-valued logic second-order vagueness continuum-valued logic rationale continuum-valued logic truth-tables
1,viii Contents\n4.9\n4.10\n4.11\n4.12\n4.13\n4.14\n4.15\n\nFuzzy sets and fuzzy logic\nDegree-theoretic treatments of sorites paradoxes\nComparatives and modifiers\nVague degrees of truth\nNon-numerical degrees of truth\nDegree-functionality\nAppendix: axiomatizations of continuum-valued logic\n\n120\n123\n124\n127\n131\n135\n138\n\n5 Supervaluations\n5.1 Incomplete meanings\n5.2 Origins\n5.3 Logic and semantics\n5.4 The elusiveness of supertruth\n5.5 Supervaluational degrees of truth\n5.6 Supervaluations and higher-order vagueness\n5.7 Truth and supertruth\n\n142\n142\n143\n146\n153\n154\n156\n162\n\n6 Nihilism\n6.1 Despair\n6.2 Global nihilism\n6.3 Local nihilism: appearances\n6.4 Local nihilism: colours\n\n165\n165\n166\n171\n180\n\n7 Vagueness as ignorance\n7.1 Bivalence and ignorance\n7.2 Bivalence and truth\n7.3 Omniscient speakers\n7.4 The supervenience of vagueness\n7.5 Meaning and use\n7.6 Understanding\n7.7 Decidable cases\n\n185\n185\n187\n198\n201\n205\n209\n212\n\n8 Inexact knowledge\n8.1 The explanatory task\n8.2 The crowd\n8.3 Margins for error\n8.4 Conceptual sources of inexactness\n8.5 Recognition of vague concepts\n8.6 Indiscriminable differences\n8.7 Inexact beliefs\n\n216\n216\n217\n226\n230\n234\n237\n244,8,8,Contents,,,viii content fuzzy set fuzzy logic degree-theoretic treatment sorites paradox comparative modifier vague degree truth non-numerical degree truth degree-functionality appendix axiomatizations continuum-valued logic supervaluations incomplete meaning origin logic semantics elusiveness supertruth supervaluational degree truth supervaluations higher-order vagueness truth supertruth nihilism despair global nihilism local nihilism appearance local nihilism colour vagueness ignorance bivalence ignorance bivalence truth omniscient speaker supervenience vagueness meaning use understanding decidable case inexact knowledge explanatory task crowd margin error conceptual source inexactness recognition vague concept indiscriminable difference inexact belief
2,Contents ix\n9 Vagueness in the world\n9.1 Supervenience and vague facts\n9.2 Determinacy in the world\n9.3 Unclarity de re\n\n248\n248\n249\n257\n\nAppendix The logic of clarity\n\n270\n\nNotes\nReferences\nIndex\n\n276\n307\n320,9,9,Contents,,,content ix vagueness world supervenience vague fact determinacy world unclarity de appendix logic clarity note reference index
3,"Preface\n\nThis 

In [840]:
with_split_data['by_page_biblio']

,content,page_number,real_page_num,section_level_1,section_level_2,section_level_3
0,"References\n\nAbbott, W.R. 1983.\n‘A note on Grim’s sorites argument’, Analysis 43: 161–4.\nAldrich, V. 1937.\n‘Some meanings of “vague”’, Analysis 4: 89–95.\nAlston, W.P. 1964.\nPhilosophy of Language, Englewood Cliffs, N.J., Prentice-Hall.\nAnnas, J. 1990.\n‘Stoic epistemology’, in S. Everson (ed.)\nCompanions to Ancient\nThought 1: Epistemology, Cambridge, Cambridge University Press.\nÅqvist, L. 1962.\n‘Reflections on the logic of nonsense’, Theoria 28: 138–57.\nArmstrong, S.L., Gleitman, L.R. and Gleitman, H. 1983.\n‘What some concepts might not be’, Cognition 13: 263–308.\nAustin, J.L. 1962.\nSense and Sensibilia, ed.\nG.J. Warnock, Oxford, Oxford\nUniversity Press.\nAyer, A.J. 1940.\nThe Foundations of Empirical Knowledge, London, Macmillan.\nBain, A. 1870.\nLogic, London, Longman.\nBaldwin, J.M. 1901–2.\nDictionary of Philosophy and Psychology, 2 vols, London,\nMacmillan.\nBarnes, J. 1982.\n‘Medicine, experience and logic’, in J. Barnes, J. Brunschwig, M.F.\nBurnyeat and M. Schofield (eds) Science and Speculation, Cambridge,\nCambridge University Press.\nBellman, R.E. and Zadeh, L.A. 1977.\n‘Local and fuzzy logics’, in J.M. Dunn and G.\nEpstein (eds) Modern Uses of Multiple-Valued Logic, Dordrecht, Reidel.\nBenjamin, A.C. 1939.\n‘Science and vagueness’, Philosophy of Science 6: 422–31.\nBeth, E.W. 1959.\nThe Foundations of Mathematics, Amsterdam, North-Holland.\nBlack, M. 1937.\n‘Vagueness: an exercise in logical analysis’, Philosophy of Science\n4: 427–55; reprinted in his Language and Philosophy, Ithaca, N.Y., Cornell\nUniversity Press, 1949.\nBlack, M. 1949.\nLanguage and Philosophy, Ithaca, N.Y., Cornell University Press.\nBlack, M. 1963.\n‘Reasoning with loose concepts’, Dialogue 2: 1–12; reprinted in his Margins of Precision, Ithaca, N.Y., Cornell University Press, 1970.\nBlack, M. 1970.\nMargins of Precision, Ithaca, N.Y., Cornell University Press.\nBloch, M. 1954.\nThe Historian’s Craft, tr.\nP.Putnam, Manchester, Manchester\nUniversity Press.",321,321,References,,
1,"308 References\nBochvar, D.A. 1981.\n‘On a three-valued logical calculus and its application to the analysis of the paradoxes of the classical extended functional calculus’, tr.\nM.\nBergmann, History and Philosophy of Logic 2: 87–112.\nBradley, F.H. 1893.\nAppearance and Reality, London, George Allen.\nBridge, J. 1977.\nBeginning Model Theory, Oxford, Clarendon Press.\nBrock, J. 1979.\n‘Principal themes in Peirce’s logic of vagueness’, Peirce Studies 1:\n41–9.\nBrock, J. 1980.\n‘Peirce’s anticipation of game-theoretic logic and semantics’, in M.\nHerzfield and M.D. Lenhart Semiotics 1980, New York, Plenum.\nBroome, J. 1984.\n‘Indefiniteness in identity’, Analysis 44: 6–12.\nBurge, T. 1990.\n‘Frege on sense and linguistic meaning’, in D. Bell and N. Cooper\n(eds) The Analytic Tradition, Oxford, Blackwell.\nBurgess, J.A. 1989.\n‘Vague identity: Evans misrepresented’, Analysis 49: 112–19.\nBurgess, J.A. 1990a.\n‘Vague objects and indefinite identity’, Philosophical Studies\n59: 263–87.\nBurgess, J.A. 1990b.\n‘The sorites paradox and higher-order vagueness’, Synthese\n85: 417–74.\nBurgess, J.A. and Humberstone, I.L. 1987.\n‘Natural deduction rules for a logic of vagueness’, Erkenntnis 27: 197–229.\nBurks, A.W. 1946.\n‘Empiricism and vagueness’,Journal of Philosophy 43: 477–86.\nBurns, L.C. 1986.\n‘Vagueness and coherence’, Synthese 68: 487–513.\nBurns, L.C. 1991.\nVagueness: An Investigation into Natural Languages and the\nSorites Paradox, Dordrecht, Kluwer.\nBurnyeat, M.F. 1982.\n‘Gods and heaps’, in M. Schofield and M.C. Nussbaum (eds)\nLanguage and Logos, Cambridge, Cambridge University Press.\nCampbell, R. 1974.\n‘The sorites paradox’, Philosophical Studies 26: 175–91.\nCargile, J. 1969.\n‘The sorites paradox’, British Journal for the Philosophy of\nScience 20: 193–202.\nCargile, J. 1979.\nParadoxes: A Study in Form and Predication, Cambridge,\nCambridge Universi

In [841]:
with_split_data['by_page_index']

,content,page_number,real_page_num,section_level_1,section_level_2,section_level_3
0,"Index\n\nAbbott, W.R. 298 accuracy 61–5, 284 admissible interpretation 144, 145,\n158–60, 294, 296, 297\nAldrich, V. 279, 285\nAlexander of Aphrodisias 278\nAlston, W.P. 279, 285 ambiguity 43, 46, 66, 73, 197, 198,\n276, 283, 285, 286 analytic–synthetic distinction 91,\n287\nAnnas, J. 278\nApollonius Cronus 10\nÅqvist, L. 289\nArcesilaus 11 argument by cases 151, 152, 164,\n295–7\nAristotle 8–10, 12, 30–2, 188,\n276\nArmstrong, S.L. 292\nAspasius 30 assent 14–22, 26 assertibility, warranted 14, 21,\n178, 200, 201, 207\nAugustine, St 279\nAustin, J.L. 71, 72, 279, 284, 285\nAyer, A.J. 284, 287\n\nBaldwin, J.M. 281\nBarnes, J. 18, 20, 22, 276–8 belief, inexact 217, 244–7, 303\nBellman, R.E. 293\nBenjamin, A.C. 285\nBergson, H. 53\nBeth, E.W. 276 bivalence 1, 2, 5, 7, 12, 13, 23,\n27, 34, 42, 74, 95, 96, 98, 99, 102,\n145, 163, 185–98, 277, 293, 294,\n299–301\nBlack, M. 5, 70, 73–83, 95, 103,\n282, 285–8, 301\nBloch, M. 137, 293\nBochvar, D.A. 288\nBradley, F.H. 303\nBridge, J. 283\nBrock, J. 281, 282\nBroome, J. 304\nBrouwer, L.E.J. 287\nBrouwersche schema see B\nBrouwersche system see KTB\nBurge, T. 280\nBurgess, J.A. 291, 296, 304\nBurks, A.W. 251, 252, 285, 286,\n303\nBurns, L.C. 299, 301, 303\nBurnyeat, M.F. 30, 276–8\n\nB (bivalence) 188; see also bivalence\nB (Brouwersche schema) 271,\n272, 297\nB (Brouwersche system) see KTB\nBain, A. 34, 279\n\nCaesarius, J. 279\nCallimachus 24\nCampbell, R. 287, 300\nCargile, J. 300\nCarnap, R. 294",334,334,Index,,
1,"Index 321\nCarneades 27, 28\nCarruthers, P. 281\nCasari, E. 291 causal theory of reference 209,\n298\nChellas, B.F. 305\nChisholm, R.M. 287\nChomsky, N. 73\nChrysippus 12, 13, 15, 16, 19–22,\n24, 25, 27–9, 276–8\nCicero 12, 13, 32, 33, 276–9 clarity 2, 6, 11, 14–22, 27, 142,\n259–68, 270–5, 277, 305\nCleanthes 12\nCleave, J.P. 289 cognitive impressions 11–20, 277,\n278\nCohen, M.R. 284 colours 180–4, 299 comparatives 124–7, 155, 156,\n291, 295 compositionality, semantic 39–46,\n54–7, 97–102, 104, 108, 110, 111,\n114–20, 122, 124–7, 132–8, 145–\n7, 153, 154, 168, 189, 190, 205,\n283, 291, 295, 299, 301 conditional proof 151, 152, 164,\n295, 297 consistency of application 74–83,\n285, 286 consistency profile 75–83, 285,\n286 context dependence 34, 214, 215,\n281, 287, 291, 301 contraposition 151, 152, 164,\n295, 297\nCook, M. 304\nCopilowish, I.M. 285\nCowles, D.W. 304\nCresswell, M.J. 291, 305, 306 cut rule 23\nDay, T.J. 297\nDeas, R. 279 definiteness operator 119,\n149–52, 157–61, 164, 194, 195,\n199, 292, 297, 304 definition 34, 38–43, 143, 144,\n\n158, 279 degrees 5, 13, 30, 74, 75, 77, 80,\n81, 96, 97, 102, 113–41, 147, 154–\n6, 252, 276, 286–93, 295, 303 de re – de dicto distinction 259–62,\n267, 268, 304, 305\nDescartes, R. 11\nDiodorus Cronus 10, 11, 23, 24\nDiogenes Laertius 276–8 disquotational property of truth\n162, 163, 188, 190–2, 298,\n300\nDubois, D. 291\nDuhem, P.M.M. 77\nDummett, M.A.E. 43, 146, 250, 280,\n288, 294, 295, 297–301,\n303\nEdgington, D. 295\nEngel, P. 292, 304\nEngel, R.E. 291\nEngel-Tiercelin, C. 281 epistemic theory of vagueness 3,\n4, 6, 13–24, 26–9, 34, 38, 39, 53,\n73, 164, 185–217, 230–7, 244–7, `\n249, 257, 263–9, 300, 301\nEubulides 8–10, 30, 277\nEvans, M.G.J 253–6, 265, 280,\n300, 304 evidence 245–7 excluded middle, law of 9, 35,\n41, 51, 58, 59, 76, 91, 92, 99,\n102, 103, 105, 112, 118, 120, 129,\n130, 145, 149, 189, 201, 280, 287,\n292, 299, 300\nF (disquotation for falsity)\n188 family resemblances 85–9\nField, H. 294\nFine, K. 146, 161, 163, 283,\n293–7\nFisch, M.H. 288 fixed margin model 270\nForbes, G. 291, 293, 295\nFraassen, B. van 146, 294\nFrede, M. 276, 277\nFrege, G. 5, 37–46, 59, 83, 86,",335,335,Index,,
2,"322 Index\n96, 98, 104, 165, 210, 262, 279,\n280\nFuhrmann, G. 292 future tense 2, 12, 102, 156, 157,\n288 fuzzy logic 5, 122–4, 139, 140,\n290, 291 fuzzy semantics 124–7, 291 fuzzy sets 120–2, 290\nGalen 28, 29 games, semantic 49–51,

In [817]:


split_data = load_split_data(
    processed_data_dir_path=PROCESSED_DATA_DIR_PATH,
    pdf_filepath=file_path
  )
# split_data['by_line_toc'].head()

(candidates_df, freq_ngrams) = get_candidates_and_frequencies(split_data)

# for file_path in file_paths[3:4]:
#   split_data = load_split_data(
#     processed_data_dir_path=PROCESSED_DATA_DIR_PATH,
#     pdf_filepath=file_path
#   )
#   split_data['by_line_toc'].head()

#   (candidates_df, freq_ngrams) = get_candidates_and_frequencies(split_data)

#   candidates_df.head()

#   with_frequencies = add_frequencies_column(
#     by_pages_body_df=split_data['by_page_body'],
#     candidates_df=candidates_df,
#     freq_ngrams=freq_ngrams
#   )

#   with_frequencies.head()

#candidates_df.head(30)

In [818]:
with_frequencies = add_frequencies_column(
  by_pages_body_df=split_data['by_page_body'],
  candidates_df=candidates_df,
  freq_ngrams=freq_ngrams
)

In [819]:
with_is_in_toc = add_is_in_toc(
  candidates_df=with_frequencies, 
  by_line_toc=split_data['by_line_toc']
)

#with_is_in_toc.head(30)


In [820]:
with_position_in_context = add_position_in_context(with_is_in_toc)



In [821]:
with_importance = add_importance(with_position_in_context)

In [822]:
with_is_named_entity = add_is_named_entity(
  candidates_df=candidates_df,
  df_pages_body=split_data['by_page_body']
)

In [823]:
with_length_of_word = add_length_of_word(candidates_df)

In [824]:
with_is_named_author = add_is_named_author(
  candidates_df=with_length_of_word,
  df_pages_biblio=split_data['by_page_biblio']
)

In [825]:
with_tfidf = add_tfidf(
  candidates_df=with_is_named_author,
  df_pages_body=split_data['by_page_body']
)

In [826]:
with_tfidf.head()

,candidate_keyword,clean_context,raw_context,POS,freq,is_in_toc,position_in_context,importance,is_named_entity,length,is_named_author,tfidf
0,introduction,introduction meaning reference,introduction meaning and reference,NOUN,0.000128,1,0.000000,0.128918,1,12,0,0.048465
1,meaning,introduction meaning reference,introduction meaning and reference,NOUN,0.008093,1,0.333333,0.128918,1,7,0,0.438762
2,reference,introduction meaning reference,introduction meaning and reference,NOUN,0.001368,1,1.000000,0.128918,1,9,0,0.525783
3,introduction meaning,introduction meaning reference,introduction meaning and reference,CHUNK,0.000038,0,0.000000,0.128918,0,20,0,0.000000
4,meaning reference,introduction meaning reference,introduction meaning and reference,CHUNK,0.000077,0,0.333333,0.128918,0,17,0,0.000000


In [827]:
(raw_indexes_list, updated_by_line_index) = get_raw_indexes_list(
  df_cann_lines_index=split_data['by_line_index']
)

In [828]:
raw_indexes_list

[[''],
 ['assignment function'],
 ['abbot', 'b', 'abstract entity'],
 ['ackerman', '', 'f', 'action sentence', 'actual referent'],
 ['adam', 'e'],
 ['almog', 'j'],
 ['alston', 'w', 'ambiguity'],
 ['ambiguous name'],
 ['amis',
  'k',
  'analogical meaning shift',
  'analytic sentence',
  'anaphora',
  'antecedent',
  'anaphor',
  'arithmetical truth',
  'asserting'],
 ['assignment function', 'attributive use', 'definite description'],
 ['referential', 'use definite', 'description'],
 ['austin', 'j', 'l'],
 ['avramides', ''],
 ['ayer', '', 'j'],
 ['bach', 'k'],
 ['bar', '', ''],
 ['barker', ''],
 ['beardsley', '', 'behaviorism', 'belief sentence'],
 ['belnap', 'n'],
 ['bennett', 'j'],
 ['bezuidenhout', ''],
 ['biro', 'j'],
 ['black', ''],
 ['blackburn', ''],
 ['blakemore', ''],
 ['bo', 'r', ''],
 ['bradley', 'f', 'h'],
 ['bradley', 'regress'],
 ['brandom', 'r'],
 ['braun', ''],
 ['brogaard', 'b'],
 ['burge', ''],
 ['camp', 'j', 'cancelability implicature'],
 ['carnap', 'r'],
 ['carnie', 

In [829]:
save_raw_indexes_list(
  processed_data_dir_path=PROCESSED_DATA_DIR_PATH,
  pdf_filepath=file_path,
  raw_indexes_list=raw_indexes_list
)

In [830]:
# clean_indexes


In [831]:
clean_indexes = load_raw_indexes_list(
  processed_data_dir_path=PROCESSED_DATA_DIR_PATH,
  pdf_filepath=file_path
)

In [832]:
with_is_in_index = add_is_in_index(
  candidates_df=with_tfidf,
  indexes_list=clean_indexes
)

In [833]:
aggregated_df=aggregate_by_candidate(
  candidates_df=with_is_in_index  
)

In [834]:
print(len(aggregated_df.candidate_keyword.unique()))
aggregated_df.shape[0]

34499


34499

In [835]:
#CHANGEEEEE
save_aggregated_data(
    processed_data_dir_path=PROCESSED_DATA_DIR_PATH,
    agg_df=aggregated_df,
    file_name="williamson_94.pdf"
)

In [836]:
aggregated_df[aggregated_df.is_in_index==1].head(100)

,candidate_keyword,length,is_named_entity,is_named_author,is_in_toc,freq,is_in_index,tfidf,importance,position_in_context,POS
90,abstract entity,15,0,0,0,0.000141,1,0.000000,0.427338,0.523066,CHUNK
273,ackerman,8,1,1,0,0.000026,1,0.016358,0.478252,0.088462,PROPN
368,action sentence,15,0,0,0,0.000013,1,0.000000,0.401335,0.642857,CHUNK
418,actual referent,15,0,0,0,0.000153,1,0.000000,0.413900,0.600042,CHUNK
472,adam,4,0,0,0,0.000013,1,0.008609,0.518424,0.125000,PROPN
846,almog,5,0,1,0,0.000013,1,0.007895,0.370415,0.000000,PROPN
1007,alston,6,1,1,0,0.000077,1,0.046765,0.436427,0.248165,PROPN
1100,ambiguity,9,0,0,0,0.000179,1,0.021171,0.415764,0.670279,NOUN
1118,ambiguous name,14,0,0,0,0.000051,1,0.000000,0.484890,0.550212,CHUNK
1147,amis,4,1,0,0,0.000038,1,0.055941,0.423431,0.377289,PROPN
